### Market data refresh 

### Input Description

RAW OHLC data.

### Output  

Clean OHLC data in a hdf store

### Operations

This code takes a financial market data file and runs it through a processing pipeline. The following operations are carried out :

- Localise the time data to market time
- Merge with existing RAW data based on datetime
- Save the resulting RAW data to HDF5

In [24]:
#!pip install --upgrade "../../quantutils"
!pip install "../../marketinsights-price-aggregator"
import json, os, pandas
import quantutils.dataset.pipeline as ppl
from quantutils.api.datasource import MarketDataStore
import MIPriceAggregator.connectors as connectors


Processing /home/cwilkin/Development/repos/marketinsights-price-aggregator
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/cwilko/quantutils.git to /tmp/pip-install-szy6xnxp/quantutils_3d78a5b9ed0b430094f52b3d2397eaa1
  Running command git clone --filter=blob:none -q https://github.com/cwilko/quantutils.git /tmp/pip-install-szy6xnxp/quantutils_3d78a5b9ed0b430094f52b3d2397eaa1
  Resolved https://github.com/cwilko/quantutils.git to commit 1f6c9a799764ce6f05219d9a473fad378bbbc463
  Preparing metadata (setup.py) ... done
  Created wheel for MIPriceAggregator: filename=MIPriceAggregator-1.0.0-py3-none-any.whl size=2766 sha256=0004dc2a247839bf881070f38e9a0f8e8562b50fa6dee7a6eea7dbf73f29e207
  Stored in directory: /home/cwilkin/.cache/pip/wheels/a3/66/dc/3a75635dd1cbb1bf931d0df38e458b1890be1666bbb5b88c7f
Successfully built MIPriceAggregator
  Attempting uninstall: MIPriceAggregator
    Found existing installation: MIPriceAggregator 1.0.0
    Uninstalling MIPriceAggregato

In [7]:
def getConnector(connClass, connName, tz, options):
    connectorClass = getattr(connectors, connClass)
    connectorInstance = connectorClass(connName, tz, options)
    return connectorInstance

def refreshDatasources(mds, ds_file):

    # Loop over datasources...
    # TODO: In chronological order

    datasources = json.load(open(ds_file))
    
    for datasource in datasources:

        options = datasource["options"]        
        dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], options)

        for market in datasource["markets"]:

            for source in market["sources"]:
                
                newData = dataConnector.getData(source)
                
                print("Adding " + datasource["name"] + " to " + market["name"] + " table")
                mds.append(source["name"], newData, source["sample_unit"], update=True)


In [10]:
# Local
mds = MarketDataStore(location="../datastore")
refreshDatasources(mds, "../datasources/datasources_todo.json")

In [31]:
# Remote (cluster)
mds = MarketDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
refreshMarketData(mds, "../datasources", "datasources.json")

Adding WallSt-hourly-120217.txt to DOW table
Converting from Europe/London to UTC
http://pricestore.192.168.1.203.nip.io/prices/datasource/WallSt-hourly?unit=H
<Response [200]>
{"rc":"success"}

{'rc': 'success'}
http://pricestore.192.168.1.203.nip.io/prices/datasource/WallSt-hourly?unit=H
{'rc': 'success'}
Adding WallSt-hourly-011116.txt to DOW table
Converting from Europe/London to UTC
http://pricestore.192.168.1.203.nip.io/prices/datasource/WallSt-hourly?unit=H
<Response [200]>
{"rc":"success"}

{'rc': 'success'}
http://pricestore.192.168.1.203.nip.io/prices/datasource/WallSt-hourly?unit=H
{'rc': 'success'}
Adding WallSt-hourly-210618.txt to DOW table
Converting from Europe/London to UTC
http://pricestore.192.168.1.203.nip.io/prices/datasource/WallSt-hourly?unit=H
<Response [200]>
{"rc":"success"}

{'rc': 'success'}
http://pricestore.192.168.1.203.nip.io/prices/datasource/WallSt-hourly?unit=H
{'rc': 'success'}
Adding WallSt-hourly-230718.txt to DOW table
Converting from Europe/Londo

http://pricestore.192.168.1.203.nip.io/prices/datasource/SP500-hourly?unit=H
<Response [200]>
{"rc":"success"}

{'rc': 'success'}
http://pricestore.192.168.1.203.nip.io/prices/datasource/SP500-hourly?unit=H
{'rc': 'success'}
Adding SP500-hourly-230718.txt to SPY table
Converting from Europe/London to UTC
http://pricestore.192.168.1.203.nip.io/prices/datasource/SP500-hourly?unit=H
<Response [200]>
{"rc":"success"}

{'rc': 'success'}
http://pricestore.192.168.1.203.nip.io/prices/datasource/SP500-hourly?unit=H
{'rc': 'success'}
Adding SP500-hourly-050517.txt to SPY table
Converting from Europe/London to UTC
http://pricestore.192.168.1.203.nip.io/prices/datasource/SP500-hourly?unit=H
<Response [200]>
{"rc":"success"}

{'rc': 'success'}
http://pricestore.192.168.1.203.nip.io/prices/datasource/SP500-hourly?unit=H
{'rc': 'success'}
Adding SP500-hourly-050617.txt to SPY table
Converting from Europe/London to UTC
http://pricestore.192.168.1.203.nip.io/prices/datasource/SP500-hourly?unit=H
<Resp

In [ ]:
# Remote (localhost)
mds = MarketDataStore(remote=True, location="http://localhost:8080")
refreshMarketData(mds, "../datasources", "datasources.json")

In [19]:
df1 = mds.aggregate("1979-01-01", "2050-01-01",  ["WallSt-hourly","D&J-IND"], "H")

Loading data from WallSt-hourly in ../tmp2/data.hdf
Resampling to H periods
Resampling to H periods
Loading data from D&J-IND in ../tmp2/data.hdf
Resampling to H periods
Resampling to H periods
